## Convolutional Nets in Python/NumPy

### network overview

| layer   | description    | length     | dimensions(shape)                   | hyperparameters                      | 
|:-------:|:--------------:|:----------:|:-----------------------------------:|:------------------------------------:|
| Layer 1 | Input          | colorbar: 3| 3x32x32                             |                                      |
| Layer 2 | Convolutional  | \*32       | weights(32x3x5x5), output(32x32)    | fieldsize=\*5, stride=\*1, pad=2     |
| Layer 3 | Pooling        | 32         | output(16x16)                       | fieldsize=\*2, stride=\*2, pad=0     |
| Layer 4 | Convolutional  | \*64       | weights(64x32x5x5), output(16x16)   | (same as layer 2)                    |
| Layer 5 | Pooling        | 64         | output(8x8)                         | (same as layer 3)                    |
| Layer 6 | Convolutional  | \*128      | weights(128x64x5x5), output(8x8)    | (same as layer 2)                    |
| Layer 7 | Pooling        | 128        | output(4x4)                         | (same as layer 3)                    |
| Layer 8 | Fully-Connected|            | input (2048)  output (\*256)        |                                      | 
| Layer 9 | Output         |            | input (256)  output(10)             |                                      |
|         |                |            |                                     | \* value set by user                 |

This notebook is coded in Python/NumPy without the use of Tensforflow or any other deep learning library (with the exception<br>of sklearn, for shuffling of 3D image data), with an intention for better understanding the details that pertain to convolutional<br>neural networks, including convolutional and pooling functions, activation functions (ReLU, Softmax and SVM), forward and <br>backward propogation with batch normalization and dropout, gradient check, mini-batch processing, and optimization<br>methods including SGD, SGD with momentum, RMS Prop, and Adam.<br><br>Contents (sections)
<br>----------------------<br>
model, batch and data classes<br>
convolution and pooling functions, 'im2col'<br>
activation and delta functions<br>
batch normalization functions<br>
forward and backward pass functions<br>
gradient check<br>
training (resumed)<br>
optimization methods<br>
&nbsp;&nbsp;&nbsp;&nbsp;notes on various methods<br>
&nbsp;&nbsp;&nbsp;&nbsp;adam optimization<br>
appendix<br>
&nbsp;&nbsp;&nbsp;&nbsp;chart of forward/backward propogation<br>
&nbsp;&nbsp;&nbsp;&nbsp;derivation of backpropogation formulas<br>
&nbsp;&nbsp;&nbsp;&nbsp;notes pertaining to use of 'batchnorm' vs 'not using batchnorm'

In [1]:
# dependencies
import numpy as np
import pickle
import os
from sklearn.utils import shuffle as skshuffle

### model class

In [2]:
class Model:
    
    def __init__(self):       

        # weights/filters - number assigned according to convolutional layer #
        # dimensions (depthcolumn(L), depthcolumn(L-1), ksize, ksize) 
        self.ksize    = 5   # kernel size of weight filter / receptive field
        np.random.seed(1)
        self.W2       = 0.1 * np.random.randn( 32,  3, self.ksize, self.ksize) 
        self.W4       = 0.1 * np.random.randn( 64, 32, self.ksize, self.ksize) 
        self.W6       = 0.1 * np.random.randn(128, 64, self.ksize, self.ksize) 
        
        # fully-connected weights (number assigned according to fully-connected layer #)
        self.W8       = 0.1 * np.random.randn(2048, 256)  # input size,  hidden size
        self.W9       = 0.1 * np.random.randn(256,  10)   # hidden size, output size
        
        # batchnorm parameters: 'gamma'(scale parameter), 'beta'(shift parameter) 
        self.gam2     = np.ones(  ( 32,32,32) )           # per spatial location 
        self.gam4     = np.ones(  ( 64,16,16) )           # "    
        self.gam6     = np.ones(  (128, 8, 8) )           # "
        self.gam8     = np.ones(  (1, self.W8.shape[1]) ) # per hidden neuron     
        self.gam9     = np.ones(  (1, self.W9.shape[1]) ) # per output neuron      
        self.beta2    = np.zeros( ( 32,32,32) )        
        self.beta4    = np.zeros( ( 64,16,16) )      
        self.beta6    = np.zeros( (128, 8, 8) )      
        self.beta8    = np.zeros( (1, self.W8.shape[1]) )     
        self.beta9    = np.zeros( (1, self.W9.shape[1]) )     
        
        #batchnorm variables: mean and variance moving averages
        self.mu2      = np.zeros( ( 32,32,32) )            
        self.mu4      = np.zeros( ( 64,16,16) )             
        self.mu6      = np.zeros( (128, 8, 8) )            
        self.mu8      = np.zeros( (1, self.W8.shape[1]) )            
        self.mu9      = np.zeros( (1, self.W9.shape[1]) )             
        self.var2     = np.zeros( ( 32,32,32) )                
        self.var4     = np.zeros( ( 64,16,16) )             
        self.var6     = np.zeros( (128, 8, 8) )            
        self.var8     = np.zeros( (1, self.W8.shape[1]) )             
        self.var9     = np.zeros( (1, self.W9.shape[1]) ) 
        
        # misc
        self.lmbda    = 1e-03     # L2 regularization penalty
        self.numW     = 5         # number of layers with weights

### batch class 
class used for mini-batches, validation & testing, and gradient check

In [3]:
class Batch:  
    def __init__(self, size=128):
        self.images   = []                                                      # CIFAR data        
        self.labels   = []                                # actual classes as binary vectors
        self.Y        = []                                      # actual classes as integers
        self.size     = size                   # size of mini-batch or validation/test batch
        self.imap     = list( range(self.size) )                # index of training examples
        
        # variables stored during forward pass
        self.Zmu2     = []   # dot product activity prior to normalization, minus batch mean
        self.Zmu4     = []   # "
        self.Zmu6     = []   # "
        self.Zmu8     = []   # "
        self.Zmu9     = []   # "
        self.Zn2      = []                                #  dot product activity normalized
        self.Zn4      = []                                #  "
        self.Zn6      = []                                #  " 
        self.Zn8      = []                                #  " 
        self.Zn9      = []                                #  "  
        self.sdi2     = []                                      # standard deviation inverse
        self.sdi4     = []                                      # "
        self.sdi6     = []                                      # "
        self.sdi8     = []                                      # "
        self.sdi9     = []                                      # "
        self.imgshp   = (self.size,   3, 32, 32)                               # input shape   
        self.A3shape  = (self.size,  32, 16, 16)             # shape of pooling layer output
        self.A5shape  = (self.size,  64,  8,  8)             # "                   
        self.A7shape  = (self.size, 128,  4,  4)             # "  
        self.A7flat   = np.zeros( (self.size, 2048) )    # output flattened by train-example  
        self.A8       = []                  # ReLU activated output to fully-connected layer
        self.Xcol2    = []      # 'im2col' aligned data, associated with convolutional layer
        self.Xcol4    = []      #  "
        self.Xcol6    = []      #  "
        self.Xcol3    = []            # 'im2col' aligned data, associated with pooling layer
        self.Xcol5    = []            #  "
        self.Xcol7    = []            #  "
        self.U8       = []                # dropout mask applicable to fully-connected layer
        self.U9       = []                # "        
        self.yHat     = []                          # Softmax activated output probabilities
        self.SVMloss  = []                                   # loss matrix applicable to SVM
        
        # parameters applicable to convolutional layers 2, 4 and 6  
        self.stride   =  1                    # number of pixels moved by filter at one time
        self.pad      =  2       # amount of zero padding to surround border of input volume

### data class,  cifar classes
data class is used to store train, test and validation sets

In [4]:
class Data: 
    def __init__(self): 
        '''Training, Testing and Validation sets.'''
        self.XTrn     = []
        self.yTrn     = []
        self.XTst     = []
        self.yTst     = []
        self.XVal     = []
        self.yVal     = [] 

class CifarLoader(object):
    def __init__(self, source_files):
        self._source = source_files
        self._i = 0
        self.images = None
        self.labels = None
        
    def load(self):
        data = [unpickle(f) for f in self._source]
        images = np.vstack([d["data"] for d in data])
        n = len(images)
        self.images = images.reshape(n, 3, 32, 32).astype(float) / 255
        self.labels = one_hot(np.hstack([d["labels"] for d in data]), 10)
        return self

class CifarDataManager(object):
    def __init__(self):
        self.train = CifarLoader(["data_batch_{}".format(i) for i in range(1, 6)]).load()
        
def one_hot(vec, vals=10):
    n = len(vec)
    out = np.zeros((n, vals))
    out[range(n), vec] = 1
    return out

def unpickle(file):
    with open(os.path.join(DATA_PATH, file), 'rb') as fo:
        u = pickle._Unpickler(fo)   
        u.encoding = 'latin1'
        dct = u.load()
    return dct
        
def loadCifar(d):
    images, labels = skshuffle(c.train.images, c.train.labels)
    d.XTrn, d.yTrn = c.train.images[:47952],      c.train.labels[:47952]
    d.XTst, d.yTst = c.train.images[47952:48976], c.train.labels[47952:48976]
    d.XVal, d.yVal = c.train.images[48976:],      c.train.labels[48976:]

DATA_PATH = "CIFAR10_data/"
c = CifarDataManager()
d = Data
loadCifar(d)
print ('All images will be available for random batch training, except those')
print ('allocated for testing and validation.')
print ('X_train shape :', d.XTrn.shape, '     y_train shape :', d.yTrn.shape)
print ('X_test shape  : ', d.XTst.shape, '     y_test shape  : ', d.yTst.shape)
print ('X_val shape   : ', d.XVal.shape, '     y_val shape   : ', d.yVal.shape)        

All images will be available for random batch training, except those
allocated for testing and validation.
X_train shape : (47952, 3, 32, 32)      y_train shape : (47952, 10)
X_test shape  :  (1024, 3, 32, 32)      y_test shape  :  (1024, 10)
X_val shape   :  (1024, 3, 32, 32)      y_val shape   :  (1024, 10)


### convolution functions
these functions do not include a bias term (refer to appendix section: 'batchnorm' vs 'not using batchnorm')

In [5]:
def convolve_forward(b, X, W, L, test=False):
    '''Forms receptive fields from input, and performs matrix multiplication of W (flattened 
       by depth), with receptive fields (flattened by depth & training examples). 
       Input:  X - input to convolutional layer (e.g., images, pooling layer output)
               W - weights, L - network layer number    (see comments, for example shapes)'''
    Xshape0, Xshape1, Xshape2, Xshape3 = X.shape  #(example batchsize: 1000)  1000, 3, 32, 32
    Wshape0, Wshape1, Wshape2, Wshape3 = W.shape  #(example depthcolumn: 10)      10, 3, 5, 5
    Zshape2 = int( (Xshape2 - Wshape2 + 2 * b.pad) / b.stride + 1 )                      # 32
    Zshape3 = int( (Xshape3 - Wshape3 + 2 * b.pad) / b.stride + 1 )                      # 32
    Xcol = im2col_indices(X, Wshape2, Wshape3, b.pad, b.stride)           # 3*5*5, 32*32*1000
    Wcol = W.reshape(Wshape0, -1)                                                 # 10, 3*5*5
    Z = Wcol @ Xcol                                                          # 10, 32*32*1000 
    Z = Z.reshape(Wshape0, Zshape2, Zshape3, Xshape0)                      # 10, 32, 32, 1000
    if not test:   
        if L == 2: b.Xcol2 = Xcol                  
        if L == 4: b.Xcol4 = Xcol
        if L == 6: b.Xcol6 = Xcol
    return Z.transpose(3,0,1,2)                                            # 1000, 10, 32, 32

def convolve_backward(M, b, inpt, L):
    '''  Input: backpropogating error applicable to convolutional layer (L)
       Returns: gradient with respect to W, and output error(L-1)'''
    if L == 2: Xcol, W, cached_inpt_shape = b.Xcol2, M.W2, b.imgshp
    if L == 4: Xcol, W, cached_inpt_shape = b.Xcol4, M.W4, b.A3shape
    if L == 6: Xcol, W, cached_inpt_shape = b.Xcol6, M.W6, b.A5shape    
    Wshape0, Wshape1, Wshape2, Wshape3 = W.shape                                # 10, 3, 5, 5
    inpt_reshaped = inpt.transpose(1, 2, 3, 0).reshape(Wshape0, -1)             # 10, 1024000
    dW = inpt_reshaped @ Xcol.T   
    dW = dW.reshape(W.shape)
    W_reshape = W.reshape(Wshape0, -1)                                               # 10, 75
    outcol = W_reshape.T @ inpt_reshaped                                        # 75, 1024000
    outerr = col2im_indices(outcol, cached_inpt_shape, Wshape2, Wshape3,  
                            padding=b.pad, stride=b.stride)                 # 1000, 3, 32, 32
    return dW, outerr

### pooling functions 

In [6]:
def maxpool_forward(b, inpt, L, test=False, size=2, stride=2, pad=0):
    '''Input:    inpt              -  output (2D) from layer(L-1), per neuron, per example
                 L                 -  network layer number (pooling layer)
                 size, stride, pad -  preset parameters for 2x2 max pooling 
       Produces output: 1-to-1 neuron mapping from L-1 → spatial dims reduced by 50%'''
    inshape0, inshape1, inshape2, inshape3 = inpt.shape               #example: 1000,10,32,32
    outshape2 = int( (inshape2 - size) / stride + 1 )                                     #16
    outshape3 = int( (inshape3 - size) / stride + 1 )                                     #16
    in_reshaped = inpt.reshape(inshape0*inshape1, 1, inshape2, inshape3)       #10000,1,32,32
    Xcol = im2col_indices(in_reshaped, size, size, pad, stride)                    #4,2560000
    argmaxes = np.argmax(Xcol, axis=0)                                               #2560000
    outp = Xcol[argmaxes, range(argmaxes.size)]                                      #2560000
    outp = outp.reshape(outshape2, outshape3, inshape0, inshape1)              #16,16,1000,10 
    if not test:
        if L==3: b.Xcol3 = Xcol                      
        if L==5: b.Xcol5 = Xcol 
        if L==7: b.Xcol7 = Xcol 
    return outp.transpose(2,3,0,1)                                             #1000,10,16,16
               
def maxpool_backward(b, inpt, L, size=2, stride=2, pad=0):
    ''' Output_err(L) maps to Output_err(L-1).  Each value maps to position in 2x2 block.
        A(L-1) from fwd-pass was not cached, however Zn(L-1) has an identical shape.'''
    if L==7: dXcol, args, cach_in = np.zeros_like(b.Xcol7), np.argmax(b.Xcol7, axis=0), b.Zn6    
    if L==5: dXcol, args, cach_in = np.zeros_like(b.Xcol5), np.argmax(b.Xcol5, axis=0), b.Zn4
    if L==3: dXcol, args, cach_in = np.zeros_like(b.Xcol3), np.argmax(b.Xcol3, axis=0), b.Zn2
    inpt_flat = inpt.transpose(2, 3, 0, 1).ravel()                   #16,16,1000,10 → 2560000
    dXcol[args, range(args.size)] = inpt_flat                                      #4,2560000
    n, d, w, h = cach_in.shape                                                 #1000,10,32,32
    shape_ = n*d, 1, h, w                                                      #10000,1,32,32
    outerr = col2im_indices(dXcol, shape_, size, size, pad, stride)            
    if L == 7:  return outerr.reshape(cach_in.shape)
    if L == 5:  return outerr.reshape(cach_in.shape)
    if L == 3:  return outerr.reshape(cach_in.shape)                           #1000,10,32,32 

### im2col functions
supports convolution and pooling functions utilizing fancy indexing

In [7]:
def get_im2col_indices(Xshape, field_height, field_width, padding, stride):
    Xshape0, Xshape1, Xshape2, Xshape3 = Xshape
    assert (Xshape2 + 2 * padding - field_height) % stride == 0
    assert (Xshape3 + 2 * padding - field_height) % stride == 0
    output_height = int((Xshape2 + 2 * padding - field_height) / stride + 1)
    output_width = int((Xshape3 + 2 * padding - field_width) / stride + 1)
    i0 = np.repeat(np.arange(field_height), field_width)
    i0 = np.tile(i0, Xshape1)
    i1 = stride * np.repeat(np.arange(output_height), output_width)
    j0 = np.tile(np.arange(field_width), field_height * Xshape1)
    j1 = stride * np.tile(np.arange(output_width), output_height)
    i = i0.reshape(-1, 1) + i1.reshape(1, -1)
    j = j0.reshape(-1, 1) + j1.reshape(1, -1)
    k = np.repeat(np.arange(Xshape1), field_height * field_width).reshape(-1, 1)
    return (k.astype(int), i.astype(int), j.astype(int))

def im2col_indices(X, field_height, field_width, padding, stride): 
    p = padding
    X_padded = np.pad(X, ((0, 0), (0, 0), (p, p), (p, p)), mode='constant')
    k, i, j = get_im2col_indices(X.shape, field_height, field_width, padding, stride)
    cols = X_padded[:, k, i, j]
    Xshape1= X.shape[1]
    cols = cols.transpose(1, 2, 0).reshape(field_height * field_width * Xshape1, -1)
    return cols

def col2im_indices(cols, Xshape, field_height, field_width, padding, stride):
    Xshape0, Xshape1, Xshape2, Xshape3 = Xshape
    Xshape2_pd, Xshape3_pd = Xshape2 + 2 * padding, Xshape3 + 2 * padding
    X_padded = np.zeros((Xshape0, Xshape1, Xshape2_pd, Xshape3_pd), dtype=cols.dtype)
    k, i, j = get_im2col_indices(Xshape, field_height, field_width, padding, stride)
    cols_reshaped = cols.reshape(Xshape1 * field_height * field_width, -1, Xshape0)
    cols_reshaped = cols_reshaped.transpose(2, 0, 1)
    np.add.at(X_padded, (slice(None), k, i, j), cols_reshaped)
    if padding == 0:
        return X_padded
    return X_padded[:, :, padding:-padding, padding:-padding]

### activation and delta functions

In [8]:
def ReLU(z):
    return np.maximum(0, z)

def d_ReLU(outerr, z):
    '''Input:   outerr - error with respect to activity z   
                z      - activity cached from forward pass  
       Returns backpropogating error δ.'''
    backerr = outerr
    backerr[z < 0] = 0
    return backerr
    
def Leaky_ReLU(z, a=1e-3):
    '''Same as ReLU, except a small negative is returned instead of 0.'''
    return np.maximum(a*z, z) 

def d_Leaky_ReLU(outerr, z, a=1e-3):
    ''' Returns backpropogating error δ, where σ is 'Leaky_ReLU' '''
    backerr = outerr
    backerr[z < 0] *= a    
    return backerr
        
def Softmax(z):
    ''' Returns output probilities adding up to 1.0 for each training example. '''
    z = z - np.max(z)  
    return np.exp(z) / np.sum(np.exp(z), axis=1, keepdims=True)

def d_Softmax(b):
    ''' Returns output probabilities for each training example, where an adjustment is
        made to the probabilities associated with correct class: p_k − 𝟙 where y_i = k. 
        Result may be used as backpropogating error, δ. '''
    backerr = b.yHat
    backerr[b.imap, b.Y] -= 1 
    return backerr

def SVM(b, s):
    '''  Computes SVM loss matrix: "SVM_loss_i = ∑ max(0,  s_j - s_yi + d)"  
          s_j: score associated with incorrect class;   i: training example 
         s_yi: score associated with correct class;     d: margin, e.g., 1  '''
    loss_matrix = np.maximum(0,  s - s[b.imap, b.Y][:, np.newaxis] + 1) 
    loss_matrix[b.imap, b.Y] = 0 # replace correct score positions with 0
    return loss_matrix
    
def d_SVM(b):
    ''' Initialize error matrix from (cached) SVM loss. Binarize matrix by assigning
        an indicator value of 1 to the positions that did not contribute to the desired 
        margin. Sum each training example, then subtract each sum from the position 
        associated with the correct class. Result may be used as backpropogating error, δ.'''
    backerr = b.SVMloss.copy()
    backerr[backerr > 0] = 1
    backerr[b.imap, b.Y] = - np.sum(backerr, axis=1)
    return backerr

### batch normalization functions
with chart of batchnorm forward and backward steps

In [9]:
def batchnorm_forward(M, b, Z, L, test=False, const=0.9, eps=1e-8):
    ''' Batch normalization forward steps, as noted in chart.'''
    if L==2: mu, var, gamma, beta = M.mu2, M.var2, M.gam2, M.beta2  
    if L==4: mu, var, gamma, beta = M.mu4, M.var4, M.gam4, M.beta4  
    if L==6: mu, var, gamma, beta = M.mu6, M.var6, M.gam6, M.beta6 
    if L==8: mu, var, gamma, beta = M.mu8, M.var8, M.gam8, M.beta8  
    if L==9: mu, var, gamma, beta = M.mu9, M.var9, M.gam9, M.beta9
    if not test:
        N = b.size
        mu_new = 1. / N * np.sum(Z, axis=0) 
        Zmu = Z - mu_new                          
        var_new = 1. / N * np.sum(Zmu**2, axis=0)                   
        stdinv = 1. / np.sqrt(var_new + eps)                                     
        Znorm = Zmu * stdinv
        mov_mu  = const * mu  + (1. - const) * mu_new     
        mov_var = const * var + (1. - const) * var_new    
        if L==2: b.Zn2, b.Zmu2, b.sdi2, M.mu2, M.var2 = Znorm, Zmu, stdinv, mov_mu, mov_var
        if L==4: b.Zn4, b.Zmu4, b.sdi4, M.mu4, M.var4 = Znorm, Zmu, stdinv, mov_mu, mov_var
        if L==6: b.Zn6, b.Zmu6, b.sdi6, M.mu6, M.var6 = Znorm, Zmu, stdinv, mov_mu, mov_var
        if L==8: b.Zn8, b.Zmu8, b.sdi8, M.mu8, M.var8 = Znorm, Zmu, stdinv, mov_mu, mov_var
        if L==9: b.Zn9, b.Zmu9, b.sdi9, M.mu9, M.var9 = Znorm, Zmu, stdinv, mov_mu, mov_var
        return gamma * Znorm + beta
    else:
        return gamma * (Z - mu) / np.sqrt(var + eps) + beta
    
def batchnorm_backward(M, b, backerr1, L):
    '''Backward batch normalization steps, as noted in chart.''' 
    if L==9: Zn, Zmu, stdinv, gamma = b.Zn9, b.Zmu9, b.sdi9, M.gam9
    if L==8: Zn, Zmu, stdinv, gamma = b.Zn8, b.Zmu8, b.sdi8, M.gam8
    if L==6: Zn, Zmu, stdinv, gamma = b.Zn6, b.Zmu6, b.sdi6, M.gam6
    if L==4: Zn, Zmu, stdinv, gamma = b.Zn4, b.Zmu4, b.sdi4, M.gam4
    if L==2: Zn, Zmu, stdinv, gamma = b.Zn2, b.Zmu2, b.sdi2, M.gam2
    N = b.size
    dbeta = np.sum(backerr1, axis=0)                 
    dgamma = np.sum(backerr1 * Zn, axis=0)              
    dZnorm = backerr1 * gamma
    dvar = np.sum(dZnorm * Zmu, axis=0) * -.5 * stdinv**3 
    d_muA = np.sum(dZnorm * -1 * stdinv, axis=0)
    d_muB = np.mean(-2 * Zmu, axis=0) * dvar
    dZmuA = dZnorm * stdinv    
    dZmuB = 2 * Zmu * dvar / N
    dZmuC = (d_muA + d_muB) / N
    backerr2 = dZmuA + dZmuB + dZmuC
    return backerr2, dgamma, dbeta

### primary forward and backward pass functions; dropout
see appendix for chart of forward and backward propogation

In [10]:
def dropout_forward(b, Zn, L, p=0.5):
    '''Applies dropout to Z-normalized by multiplying it with droput mask, U.'''
    if L == 8:
        b.U8 = ( np.random.rand(*Zn.shape) < p ) / p
        return Zn * b.U8
    if L == 9:
        b.U9 = ( np.random.rand(*Zn.shape) < p ) / p
        return Zn * b.U9 
        
def forward(M, b, f, drp=True, test=False):
    ''' Forward steps as noted in chart (see appendix).  Input "Softmax or "SVM. '''
                # layers 2-3
    Z = convolve_forward(b, b.images, M.W2, L=2)
    Zn = batchnorm_forward(M, b, Z, L=2)
    A = ReLU(Zn)  
    A = maxpool_forward(b, inpt=A, L=3)
                # layers 4-5    
    Z = convolve_forward(b, A, M.W4, L=4)
    Zn = batchnorm_forward(M, b, Z, L=4)
    A = ReLU(Zn)  
    A = maxpool_forward(b, inpt=A, L=5)
                # layers 6-7
    Z = convolve_forward(b, A, M.W6, L=6)
    Zn = batchnorm_forward(M, b, Z, L=6)
    A = ReLU(Zn)  
    A = maxpool_forward(b, inpt=A, L=7)
    for i in range(b.size):
        b.A7flat[i] = np.ravel( A[i] )
                # layer 8        
    Z = b.A7flat @ M.W8
    Zn = batchnorm_forward(M, b, Z, L=8)
    if drp: Zn = dropout_forward(b, Zn, L=8)   
    b.A8 = ReLU(Zn)                   
                # layer 9    
    Z = b.A8 @ M.W9
    Zn = batchnorm_forward(M, b, Z, L=9)
    if drp: Zn = dropout_forward(b, Zn, L=9) 
    if f == Softmax: b.yHat = Softmax(Zn)   
    if f == SVM:     b.SVMloss = SVM(b, Zn)
        
def dropout_backward(b, backerr, L):
    '''Applies dropout to backerr1 by multiplying it with droput mask, U.'''
    if L == 9: return backerr * b.U9
    if L == 8: return backerr * b.U8
    
def backward(M, b, d_function, drp=True):
    '''Backward steps as noted in chart. Input: e.g., dSoftmax, or dSVM.'''
              # layer 9
    backerr = d_function(b)
    if drp: backerr = dropout_backward(b, backerr, L=9)  
    backerr, dgamma9, dbeta9 = batchnorm_backward(M, b, backerr, L=9)
    dW9 = b.A8.T @ backerr
    outerr = backerr @ M.W9.T
              # layer 8    
    backerr = d_ReLU(outerr, b.Zn8)
    if drp: backerr = dropout_backward(b, backerr, L=8) 
    backerr, dgamma8, dbeta8 = batchnorm_backward(M, b, backerr, L=8)
    dW8 = b.A7flat.T @ backerr
    outerr = (backerr @ M.W8.T).reshape(b.A7shape)
              # layers 7-6    
    outerr = maxpool_backward(b, outerr, L=7)
    backerr = d_ReLU(outerr, b.Zn6)
    backerr, dgamma6, dbeta6 = batchnorm_backward(M, b, backerr, L=6)    
    dW6, outerr = convolve_backward(M, b, backerr, L=6)  
              # layers 5-4   
    outerr = maxpool_backward(b, outerr, L=5)
    backerr = d_ReLU(outerr, b.Zn4)
    backerr, dgamma4, dbeta4 = batchnorm_backward(M, b, backerr, L=4)    
    dW4, outerr = convolve_backward(M, b, backerr, L=4)   
              # layers 3-2
    outerr = maxpool_backward(b, outerr, L=3)
    backerr = d_ReLU(outerr, b.Zn2)
    backerr, dgamma2, dbeta2 = batchnorm_backward(M, b, backerr, L=2)    
    dW2, outerr = convolve_backward(M, b, backerr, L=2)
    
    gradients = [dW2, dW4, dW6, dW8, dW9, dgamma2, dgamma4, dgamma6, 
                 dgamma8, dgamma9, dbeta2, dbeta4, dbeta6, dbeta8, dbeta9]
      
    return gradients

### gradient check
gradient check model is a lighter version of Model class:<br>
&nbsp;&nbsp;&nbsp;convolutional layers: &nbsp;&nbsp;layer 2: two neurons &nbsp;&nbsp;&nbsp;&nbsp;layer 4: three neurons &nbsp;&nbsp;&nbsp;&nbsp;layer 6: four neurons<br>

batch class is used - pooling activity shapes with respect to lighter network size are as follows:<br>
&nbsp;&nbsp;&nbsp;pooling layers: &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; layer 3: &nbsp; '2x16x16'&nbsp; &nbsp;&nbsp;&nbsp;&nbsp; layer 5:&nbsp;&nbsp;  '3x8x8'&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; layer 7: &nbsp;&nbsp;'4x4x4'<br><br>
note: dropout and L2 regularization are omitted from gradient check

In [11]:
class gradient_check_model:
    def __init__(self): 
        np.random.seed(1)
        self.W2              = 0.1 * np.random.randn( 2, 3, 5, 5) 
        self.W4              = 0.1 * np.random.randn( 3, 2, 5, 5) 
        self.W6              = 0.1 * np.random.randn( 4, 3, 5, 5) 
        self.W8              = 0.1 * np.random.randn( 64, 20 )  
        self.W9              = 0.1 * np.random.randn( 20, 10 )   
        self.gam2            = np.ones(  (2,32,32) )             # per spatial location 
        self.gam4            = np.ones(  (3,16,16) )             # "    
        self.gam6            = np.ones(  (4, 8, 8) )             # "
        self.gam8            = np.ones(  (1, self.W8.shape[1]) ) # per hidden neuron     
        self.gam9            = np.ones(  (1, self.W9.shape[1]) ) # per output neuron      
        self.beta2           = np.zeros( (2,32,32) )      
        self.beta4           = np.zeros( (3,16,16) )     
        self.beta6           = np.zeros( (4, 8, 8) )     
        self.beta8           = np.zeros( (1, self.W8.shape[1]) )     
        self.beta9           = np.zeros( (1, self.W9.shape[1]) )     
        self.mu2             = np.zeros( (2,32,32) )             
        self.mu4             = np.zeros( (3,16,16) )             
        self.mu6             = np.zeros( (4, 8, 8) )            
        self.mu8             = np.zeros( (1, self.W8.shape[1]) )            
        self.mu9             = np.zeros( (1, self.W9.shape[1]) )             
        self.var2            = np.zeros( (2,32,32) )             
        self.var4            = np.zeros( (3,16,16) )            
        self.var6            = np.zeros( (4, 8, 8) )            
        self.var8            = np.zeros( (1, self.W8.shape[1]) )             
        self.var9            = np.zeros( (1, self.W9.shape[1]) ) 
        self.ravelshapeW2    = self.W2.ravel().shape[0]              
        self.ravelshapegam2  = self.gam2.ravel().shape[0]
        self.ravelshapebeta2 = self.beta2.ravel().shape[0]
        self.ravelshapeW4    = self.W4.ravel().shape[0]              
        self.ravelshapegam4  = self.gam4.ravel().shape[0]
        self.ravelshapebeta4 = self.beta4.ravel().shape[0]
        self.ravelshapeW6    = self.W6.ravel().shape[0]              
        self.ravelshapegam6  = self.gam6.ravel().shape[0]
        self.ravelshapebeta6 = self.beta6.ravel().shape[0]
        self.ravelshapeW8    = self.W8.ravel().shape[0]              
        self.ravelshapegam8  = self.gam8.ravel().shape[0]
        self.ravelshapebeta8 = self.beta8.ravel().shape[0]
        self.ravelshapeW9    = self.W9.ravel().shape[0]              
        self.ravelshapegam9  = self.gam9.ravel().shape[0]
        self.ravelshapebeta9 = self.beta9.ravel().shape[0]
        
def shape_gradient_batch(b):
    b.imgshp   = (b.size,   3, 32, 32)      
    b.A3shape  = (b.size,   2, 16, 16)       
    b.A5shape  = (b.size,   3,  8,  8)                                  
    b.A7shape  = (b.size,   4,  4,  4)       
    b.A7flat   = np.zeros( (b.size, 64) )    

### gradient check (continued)

In [12]:
def setWgb(m, Wgb):
    W2_end     =         0 + m.ravelshapeW2
    gam2_end   =    W2_end + m.ravelshapegam2
    beta2_end  =  gam2_end + m.ravelshapebeta2
    W4_end     = beta2_end + m.ravelshapeW4
    gam4_end   =    W4_end + m.ravelshapegam4
    beta4_end  =  gam4_end + m.ravelshapebeta4
    W6_end     = beta4_end + m.ravelshapeW6
    gam6_end   =    W6_end + m.ravelshapegam6
    beta6_end  =  gam6_end + m.ravelshapebeta6
    W8_end     = beta6_end + m.ravelshapeW8
    gam8_end   =    W8_end + m.ravelshapegam8
    beta8_end  =  gam8_end + m.ravelshapebeta8
    W9_end     = beta8_end + m.ravelshapeW9
    gam9_end   =    W9_end + m.ravelshapegam9
    beta9_end  =  gam9_end + m.ravelshapebeta9    
    m.W2       = np.reshape( Wgb[        0:W2_end],       m.W2.shape )
    m.gam2     = np.reshape( Wgb[   W2_end:gam2_end],   m.gam2.shape )
    m.beta2    = np.reshape( Wgb[ gam2_end:beta2_end], m.beta2.shape )
    m.W4       = np.reshape( Wgb[beta2_end:W4_end],       m.W4.shape )
    m.gam4     = np.reshape( Wgb[   W4_end:gam4_end],   m.gam4.shape )
    m.beta4    = np.reshape( Wgb[ gam4_end:beta4_end], m.beta4.shape )
    m.W6       = np.reshape( Wgb[beta4_end:W6_end],       m.W6.shape ) 
    m.gam6     = np.reshape( Wgb[   W6_end:gam6_end],   m.gam6.shape )
    m.beta6    = np.reshape( Wgb[ gam6_end:beta6_end], m.beta6.shape )
    m.W8       = np.reshape( Wgb[beta6_end:W8_end],       m.W8.shape )  
    m.gam8     = np.reshape( Wgb[   W8_end:gam8_end],   m.gam8.shape )
    m.beta8    = np.reshape( Wgb[ gam8_end:beta8_end], m.beta8.shape )
    m.W9       = np.reshape( Wgb[beta8_end:W9_end],       m.W9.shape )      
    m.gam9     = np.reshape( Wgb[   W9_end:gam9_end],   m.gam9.shape )
    m.beta9    = np.reshape( Wgb[ gam9_end:beta9_end], m.beta9.shape )
    
def costFunction(m, b, f):
    forward(m, b, f, drp=False)
    if f == Softmax:
        logloss = -np.log( b.yHat[b.imap, b.Y] ) 
        return np.sum(logloss)
    if f == SVM:
        return np.sum(b.SVMloss) 
    
def computeNumericalGradients(m, b, activation, h=1e-5):
    '''Computes numerical gradient using Taylor expansion of 'f(x+h) and f(x-h)'.'''
    Wgb = np.concatenate( (m.W2.ravel(), m.gam2.ravel(), m.beta2.ravel(),
                           m.W4.ravel(), m.gam4.ravel(), m.beta4.ravel(),
                           m.W6.ravel(), m.gam6.ravel(), m.beta6.ravel(),
                           m.W8.ravel(), m.gam8.ravel(), m.beta8.ravel(),
                           m.W9.ravel(), m.gam9.ravel(), m.beta9.ravel()) )
    numerical_gradients = np.zeros_like(Wgb)
    v = np.zeros_like(Wgb)
    endrange = Wgb.shape[0] 
    for i in range(endrange):
        v[i] = h
        setWgb(m, Wgb+v)                     
        loss2 = costFunction(m, b, activation)
        setWgb(m, Wgb-v)
        loss1 = costFunction(m, b, activation)
        numerical_gradients[i] = (loss2 - loss1) / (2 * h)
        v[i] = 0
    setWgb(m, Wgb) 
    return numerical_gradients

def computeAnalyticGradients(m, b, f, delta_f):
    '''Run forward/backward pass, unpack gradients, & reformat for numerical comparison.'''
    forward(m, b, f, drp=False) 
    g = backward(m, b, delta_f, drp=False)
    dW2, dW4, dW6, dW8, dW9 = g[0],   g[1],  g[2],  g[3],  g[4] 
    dg2, dg4, dg6, dg8, dg9 = g[5],   g[6],  g[7],  g[8],  g[9]
    db2, db4, db6, db8, db9 = g[10], g[11], g[12], g[13], g[14]
    return np.concatenate( (dW2.ravel(), dg2.ravel(), db2.ravel(), dW4.ravel(), dg4.ravel(), 
                            db4.ravel(), dW6.ravel(), dg6.ravel(), db6.ravel(), dW8.ravel(), 
                            dg8.ravel(), db8.ravel(), dW9.ravel(), dg9.ravel(), db9.ravel()) )

def get_init_model(m):
    return ( [m.W2, m.W4, m.W6, m.W8, m.W9, m.gam2, m.gam4, m.gam6, m.gam8, m.gam9, 
              m.beta2, m.beta4, m.beta6, m.beta8, m.beta9]  )          

def compare_gradients(m, b, activation_function, delta_function):
    '''Computes numerical gradients, and analytic gradients, then compares both. An initial 
       parameter update is applied (since beta parameters are initialized as zeros). '''
    shape_gradient_batch(b)
    b.images, b.labels = d.XTrn[:b.size], d.yTrn[:b.size]
    b.Y = np.argmax(b.labels, axis=1)
    f, delta_f = activation_function, delta_function
    forward(m, b, f, drp=False)
    gradients = backward(m, b, delta_f, drp=False)
    mdl = get_init_model(m)  
    for param in range(len(mdl)):
        mdl[param] -= 0.001 * gradients[param]
    numgrads = computeNumericalGradients(m, b, f) 
    grads = computeAnalyticGradients(m, b, f, delta_f)
    print ('result should be on the order of 1e-8 or less:')
    print ( np.linalg.norm(grads-numgrads) / np.linalg.norm(grads+numgrads) )
    return numgrads, grads 

### gradient check (continued) -  Softmax 

In [13]:
m, b = gradient_check_model(), Batch(size=8)   
numgrads, grads = compare_gradients(m, b, Softmax, d_Softmax)

result should be on the order of 1e-8 or less:
2.23935001674e-09


### gradient check (continued) - SVM

In [14]:
m, b = gradient_check_model(), Batch(size=8)   
numgrads, grads = compare_gradients(m, b, SVM, d_SVM)

result should be on the order of 1e-8 or less:
3.8654875219e-10


### train functions
continued from primary forward and backward pass functions

In [17]:
def reg_loss(M):
    Ws = [M.W2, M.W4, M.W6, M.W8, M.W9]
    loss = 0
    for W in Ws:
        loss += 0.5 * M.lmbda * np.sum(W*W) 
    return loss

def train_step(M, b, f):
    '''A single training step: forward, loss, backprop'''
    forward(M, b, f)
    if f == Softmax:
        logloss = -np.log( b.yHat[b.imap, b.Y] ) 
        loss = np.sum(logloss) / b.size  +  reg_loss(M) / b.size
        gradients = backward(M, b, d_Softmax)
    if f == SVM:
        loss = np.sum(b.SVMloss) / b.size  +  reg_loss(M) / b.size
        gradients = backward(M, b, d_SVM)
    return gradients, loss

def accuracy(y_true, y_pred):
    return np.mean(y_pred == y_true)

def forward_val_tst(M, b):
    '''Forward steps streamlined for validation/testing.   Note: Softmax or SVM 
       activation is not required for accuracy testing.  Returns Z-normalized.'''
    Z = convolve_forward(b, b.images, M.W2, L=2, test=True)  
    Zn = batchnorm_forward(M, b, Z, L=2, test=True)
    A = maxpool_forward(b, ReLU(Zn), L=3, test=True)              
    Z = convolve_forward(b, A, M.W4, L=4, test=True)     
    Zn = batchnorm_forward(M, b, Z, L=4, test=True)
    A = maxpool_forward(b, ReLU(Zn), L=5, test=True)              
    Z = convolve_forward(b, A, M.W6, L=6, test=True)     
    Zn = batchnorm_forward(M, b, Z, L=6, test=True)
    A = maxpool_forward(b, ReLU(Zn), L=7, test=True)          
    for i in range(b.size):
        b.A7flat[i] = np.ravel( A[i])
    Z = b.A7flat @ M.W8                                  
    Zn = batchnorm_forward(M, b, Z, L=8, test=True)
    Z = ReLU(Zn) @ M.W9                                             
    return batchnorm_forward(M, b, Z, L=9, test=True)

def display_progress(d, M, b, loss, t):
    '''Makes predictions with respect to validation data, and reports accuracy based on 
       the class with highest score.'''
    b = Batch( size=len(d.XVal) ) 
    b.images, b.labels = d.XVal, d.yVal
    scores = forward_val_tst(M, b)
    val_acc = accuracy( np.argmax(b.labels, axis=1), np.argmax(scores, axis=1) )   
    print('Iter-{} loss: {:.3f} accuracy: {:3f}'.format(t, loss, val_acc))
                     
def retrieve_next_batch(d, size):
    X, y = skshuffle(d.XTrn, d.yTrn)
    images, labels = X[:size], y[:size] 
    return images, labels, np.argmax(labels, axis=1)
    
def get_init_model(M):
    return ( [M.W2, M.W4, M.W6, M.W8, M.W9, M.gam2, M.gam4, M.gam6, M.gam8, M.gam9, 
              M.beta2, M.beta4, M.beta6, M.beta8, M.beta9]  )   

# instantiate original batch class
b = Batch(size=128)
b.imgshp   = (b.size,   3, 32, 32)                  
b.A3shape  = (b.size,  32, 16, 16)            
b.A5shape  = (b.size,  64,  8,  8)                                
b.A7shape  = (b.size, 128,  4,  4)             
b.A7flat   = np.zeros((b.size, 2048))    

## optimization methods

### sgd
𝜂 &nbsp;&nbsp;&nbsp;&nbsp;:&nbsp;learnrate<br>
𝑡 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;:&nbsp;iteration<br>
∂𝑤 &nbsp;:&nbsp;gradient<br><br>
𝑤<sub>𝑡+1</sub> &nbsp; ← &nbsp; 𝑤<sub>𝑡</sub> &nbsp; - &nbsp; 𝜂 &nbsp;⋅ &nbsp;∂𝑤

### sgd with momentum
𝜌 &nbsp;: friction constant <br>
𝑣 &nbsp;: velocity, a moving average of gradients with greater emphasis on recent gradient<br>

𝑣<sub>𝑡+1</sub> &nbsp; ← &nbsp; 𝜌 &nbsp; ⋅ &nbsp;𝑣<sub>𝑡</sub>&nbsp; + &nbsp;∂𝑤<br>
𝑤<sub>𝑡+1</sub> &nbsp; ← &nbsp;𝑤<sub>𝑡</sub> &nbsp;- &nbsp;𝜂&nbsp; ⋅ &nbsp;𝑣

### rms prop
weights that receive high gradients will have their effective learning rate reduced, while weights that receive<br> small or infrequent updates will have their effective learning rate increased<br><br>
*𝛾* : decay rate constant<br>
𝑣 : cached variable that keeps track of per-parameter sum of squared gradients<br>
𝜀 : avoids division by zero<br>

𝑣<sub>𝑡+1</sub> &nbsp; ← &nbsp; *𝛾* &nbsp; ⋅ &nbsp;𝑣<sub>𝑡</sub> &nbsp;+&nbsp; (𝟙 - *𝛾*) &nbsp;⋅ &nbsp;∂𝑤<sup>2</sup><br>
𝑤<sub>𝑡+1</sub> &nbsp; ← &nbsp;𝑤<sub>𝑡</sub> &nbsp;- &nbsp;𝜂 &nbsp;/ &nbsp;( 𝑠𝑞𝑟𝑡(𝑣) &nbsp;+&nbsp; 𝜀 )

## adam optimizer
𝛽<sub>1</sub>, 𝛽<sub>2</sub>&nbsp;&nbsp;:&nbsp;decay constants<br>
𝑚 &nbsp; &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;: cached variable that keeps track of per-parameter sum of gradients&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;𝑚<sub>𝑡+1</sub> &nbsp; ← &nbsp; 𝛽<sub>1</sub> &nbsp;  ⋅ &nbsp;𝑚<sub>𝑡</sub> &nbsp;+&nbsp; ( 𝟙 - 𝛽<sub>1</sub> ) &nbsp;⋅ &nbsp;∂𝑤<br>
𝑣 &nbsp; &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;: cached variable that keeps track of per-parameter sum of squared gradients&nbsp;&nbsp;&nbsp;𝑣<sub>𝑡+1</sub> &nbsp; ← &nbsp; 𝛽<sub>2</sub> &nbsp;&nbsp;⋅ &nbsp;𝑣<sub>𝑡</sub> &nbsp; +&nbsp; ( 𝟙 - 𝛽<sub>2</sub> ) &nbsp;⋅ &nbsp;∂𝑤<sup>2</sup><br>
𝑚ℎ𝑎𝑡 &nbsp; :&nbsp;bias correction<sub>1</sub>&nbsp;&nbsp;&nbsp;&nbsp;𝑚ℎ𝑎𝑡<sub>𝑤</sub> &nbsp;←&nbsp; 𝑚<sub>𝑤</sub><sup>(𝑡+1)</sup> / ( 𝟙 - 𝛽<sub>1</sub><sup>𝑡</sup> )<br>
𝑣ℎ𝑎𝑡 &nbsp; : &nbsp;bias correction<sub>2</sub>&nbsp;&nbsp;&nbsp;&nbsp;𝑣ℎ𝑎𝑡<sub>𝑤</sub> &nbsp;←&nbsp; 𝑣<sub>𝑤</sub><sup>(𝑡+1)</sup>&nbsp; / ( 𝟙 - 𝛽<sub>2</sub><sup>𝑡</sup> )<br><br>
𝑤<sub>𝑡+1</sub> &nbsp; ← &nbsp; 𝑤<sub>𝑡</sub> &nbsp; - &nbsp; 𝜂 &nbsp; ⋅ &nbsp;𝑚ℎ𝑎𝑡 &nbsp; / &nbsp; ( 𝑠𝑞𝑟𝑡(𝑣ℎ𝑎𝑡) &nbsp;+&nbsp; 𝜀 )

In [18]:
def adam(d, M, b, f, lrate=1e-3, n_iter=2000, eps=1e-8, print_after=100):
    const1, const2 =.9, .999
    mdl = get_init_model(M) 
    m, v = np.zeros_like(mdl), np.zeros_like(mdl)
    for t in range(1, n_iter+1):
        b = Batch()
        b.images, b.labels, b.Y = retrieve_next_batch(d, b.size)
        gradients, loss = train_step(M, b, f)
        if t % print_after == 0: 
            display_progress(d, M, b, loss, t)
        for w in range(M.numW):
            gradients[w] += M.lmbda * mdl[w]
        for param in range(len(gradients)):
            m[param] = const1 * m[param] + (1. - const1) * gradients[param]
            v[param] = const2 * v[param] + (1. - const2) * gradients[param]**2
            mhat = m[param] / ( 1 - const1**(t) )    
            vhat = v[param] / ( 1 - const2**(t) )
            mdl[param] += - lrate * mhat / ( np.sqrt(vhat) + eps )
    return mdl

M = Model()
adm = adam(d, M, b, Softmax)

Iter-100 loss: 2.059 accuracy: 0.462891
Iter-200 loss: 1.957 accuracy: 0.500977
Iter-300 loss: 1.986 accuracy: 0.555664
Iter-400 loss: 1.723 accuracy: 0.540039
Iter-500 loss: 1.680 accuracy: 0.567383
Iter-600 loss: 1.655 accuracy: 0.594727
Iter-700 loss: 1.660 accuracy: 0.597656
Iter-800 loss: 1.590 accuracy: 0.573242
Iter-900 loss: 1.562 accuracy: 0.543945
Iter-1000 loss: 1.649 accuracy: 0.550781
Iter-1100 loss: 1.709 accuracy: 0.488281
Iter-1200 loss: 1.596 accuracy: 0.652344
Iter-1300 loss: 1.456 accuracy: 0.641602
Iter-1400 loss: 1.613 accuracy: 0.631836
Iter-1500 loss: 1.622 accuracy: 0.616211
Iter-1600 loss: 1.520 accuracy: 0.499023
Iter-1700 loss: 1.587 accuracy: 0.603516
Iter-1800 loss: 1.487 accuracy: 0.677734
Iter-1900 loss: 1.312 accuracy: 0.648438
Iter-2000 loss: 1.416 accuracy: 0.599609


### appendix

### chart of forward/backward steps (through layer 4)

### chart of forward/backward steps (layers 5-7)

### chart of forward/backward steps (layers 8-9)

### derivation of backpropogation formulas 

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; J &nbsp; = &nbsp; <sup>𝟷</sup>∕<sub>𝟸</sub> ⋅ ( *𝑦 - 𝑦ℎ𝑎𝑡* )<sup>𝟸</sup>


&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;∂J/∂𝑤 &nbsp; = &nbsp;&nbsp; ( *𝑦 - 𝑦ℎ𝑎𝑡* ) &nbsp;&nbsp; ⋅ &nbsp;&nbsp;   ( ∂*𝑦ℎ𝑎𝑡* / ∂𝑤 )<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp; &nbsp; = &nbsp;&nbsp; ( *𝑦 - 𝑦ℎ𝑎𝑡* ) &nbsp;&nbsp; ⋅ &nbsp;&nbsp;   ( ∂*𝑦ℎ𝑎𝑡* / ∂𝑧 ) &nbsp;&nbsp; ⋅ &nbsp;&nbsp;   ( ∂𝑧 / ∂𝑤 )<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp; &nbsp; = &nbsp;&nbsp;&nbsp;&nbsp;  𝑜𝑢𝑡𝑒𝑟𝑟<sub>𝐿</sub>  &nbsp;&nbsp;&nbsp;&nbsp;&nbsp; ⋅ &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;    σ’(𝑧<sub> 𝐿</sub> )  &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; ⋅ &nbsp;&nbsp;&nbsp;&nbsp;&nbsp; a<sub> 𝐿 - 𝟙</sub><br><br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; δ &nbsp; = &nbsp;&nbsp;&nbsp;&nbsp; 𝑜𝑢𝑡𝑒𝑟𝑟  &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; ⋅ &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;    σ’(𝑧)<br> 
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; ∂J/∂𝑤<sub>*L* </sub> &nbsp;= &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; δ<sub> *L*</sub> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; ⋅ &nbsp;&nbsp;&nbsp;&nbsp;   a<sup>T</sup><sub>*L - 𝟙*</sub><br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;𝑜𝑢𝑡𝑒𝑟𝑟<sub>*L-1*</sub> &nbsp;&nbsp; = &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; δ<sub> *L*</sub> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; ⋅ &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;   𝑤<sup>T</sup><sub>𝐿</sub><br><br>
σ is activation function<br>
σ’ is prime activation function<br>
𝑤,&nbsp; &nbsp;∂J/∂𝑤&nbsp; &&nbsp; δ&nbsp; &nbsp;do not apply to pooling layers<br>
δ is backpropgating error<br>
∂W is used for ∂J/∂𝑤 in above chart, and is referred to in code as dW 

### notes pertaining to use of 'batchnorm' vs 'not using batchnorm'
When batch normalization is applied, less focus is needed with respect to the initialization of weights. If  batchnorm <br> was not to be applied, a Xavier initialization of weights should be considered as a way of calibrating variances <br>between network layers. 

For example:<br>
    W2  = np.random.randn(32,  3,  5, 5) \* np.sqrt(2.0 / (3\*5\*5))<br>
    W4  = np.random.randn(64,  32, 5, 5) \* np.sqrt(2.0 / (32\*5\*5))<br>
    W6  = np.random.randn(128, 64, 5, 5) \* np.sqrt(2.0 / (64\*5\*5))<br>
    W8  = np.random.randn(2048, 256) \* np.sqrt(2.0 / 2048)<br>
    W9  = np.random.randn(256,  10)  \* np.sqrt(2.0 / 256)
    
ReLU is a common activation function applied in convolutional and fully-connected layers (except for output layer) <br> however, if batch normalization is not applied, 'Leaky ReLU' should be considered if a solution for dying ReLU is <br>needed.   
    
Biases are used in neural networks as shift parameters, however, if batch normalization is applied, biases may be<br> replaced by gamma and beta parameters.  Bias parameters would be coded, when applicable, as follows:<br>
* initialized for conv layer as np.zeros( (self.W{L}.shape[0], 1) ), or fc layer as np.zeros( (1, self.W{L}.shape[1]) ) 
* included in activity Z computation, e.g., Z{L} = W{L} * A{L-1} + bias{L}
* bias gradient is computed in convolution backward pass as 'db = np.sum(backerr, axis=(0, 2, 3))'
* ...and 'db.reshape(Wshape0, -1)'
* in primary backward pass function as:  'db = np.sum(backerr, axis=0, keepdims=True)'

Finally, if batch normalization is not applied, references to batchnorm forward, and batchnorm backward, as well<br> as 'Z-n' should be removed from primary forward and backward pass functions. 